In [1]:
import networkx as nx
import tsplib95 as tsp
import numpy as np
from queue import PriorityQueue

# Algorithms

## Branch and Bound

In [44]:
problem = tsp.load('test15.tsp')

In [45]:
# Implements the branch and bound algorithm

class Node:
    def __init__(self, bound, path, edges, cost):
        self.bound = bound
        self.path = path
        self.edges = edges
        self.cost = cost
    
    def __lt__(self, other):
        if len(self.path) == len(other.path): #longer paths mean we are closer to the complete solution
            return self.cost < other.cost #prioritize lower cost
        else:
            return len(self.path) > len(other.path)
    
def findSmallestEdges(problem, node):
    min1 = float('inf')
    min2 = float('inf')
    for other_node in problem.get_nodes():
        if other_node != node:
            weight = problem.get_weight(node, other_node)
            if weight < min1:
                min2 = min1
                min1 = weight
            elif weight < min2:
                min2 = weight
    return min1, min2

def computeFirstBound(problem):
    bound = 0
    num_nodes = problem.dimension
    initialBoundEdges = np.zeros((num_nodes + 1, 2))

    for i in range(1, num_nodes + 1):
        min1, min2 = findSmallestEdges(problem, i)
        print(min1, min2)
        initialBoundEdges[i][0] = min1
        initialBoundEdges[i][1] = min2
        bound += min1 + min2

    return bound / 2, initialBoundEdges

def computeNodeBound(problem, path, edges, bound):
    num_nodes = problem.dimension + 1
    changedEdges = np.zeros(num_nodes, dtype=int)
    newEdges = np.array(edges)

    # Get the weight of the most recently added edge in the path
    edgeWeight = problem.get_weight(path[-2], path[-1])
    
    # Double the current bound to adjust for calculations
    sum = bound * 2

    # Update for the second-to-last node in the path
    if newEdges[path[-2]][0] != edgeWeight:
        if changedEdges[path[-2]] == 0:
            sum -= newEdges[path[-2]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[path[-2]][0]
            sum += edgeWeight
        changedEdges[path[-2]] += 1

    # Update for the last node in the path
    if newEdges[path[-1]][0] != edgeWeight:
        if changedEdges[path[-1]] == 0:
            sum -= newEdges[path[-1]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[path[-1]][0]
            sum += edgeWeight
        changedEdges[path[-1]] += 1

    # Return the updated bound and the updated edges array
    return sum / 2, newEdges

In [46]:
def branchAndBoundTSP(problem):
    initialBound, initialBoundEdges = computeFirstBound(problem)
    root = Node(initialBound, [1], initialBoundEdges, 0)
    pq = PriorityQueue()
    pq.put(root)
    best = float('inf')
    best_solution = []
    num_nodes = problem.dimension

    while not pq.empty():
        node = pq.get()
        level = len(node.path)

        # Check if a complete solution is found
        if level == num_nodes:
            if node.cost < best:
                best = node.cost
                best_solution = node.path
        else:
            # Explore further only if the current bound is less than the best known cost
            if node.bound < best:
                for k in range(1, num_nodes + 1):
                    if k not in node.path:
                        new_solution = node.path + [k]
                        edgeWeight = problem.get_weight(node.path[-1], k)
                        newBound, newEdges = computeNodeBound(problem, new_solution, node.edges, node.bound)

                        # Check if the new node is promising
                        if newBound < best:
                            new_cost = node.cost + edgeWeight
                            newNode = Node(newBound, new_solution, newEdges, new_cost)
                            pq.put(newNode)

    # add the first node to the end of the path to complete the cycle
    best_solution.append(best_solution[0])

    # sum the weights of the last edge (from the last node to the first node) to the best cost
    best += problem.get_weight(best_solution[-2], best_solution[-1])
            
    return best, best_solution


In [47]:
best_cost, best_solution = branchAndBoundTSP(problem)
print("Best cost:", best_cost)
print("Best solution:", best_solution)

48 161
67 149
188 230
138 181
89 149
117 173
135 379
61 138
161 188
67 89
61 181
135 269
181 291
48 181
117 225
Best cost: 2884
Best solution: [1, 14, 9, 3, 2, 10, 5, 12, 7, 15, 6, 11, 8, 4, 13, 1]
